In [1]:
import pickle
import pandas as pd
import ast
from collections import OrderedDict
import unicodedata
from ftfy import fix_text
import re
import string

In [2]:
ETSY_ROUND = '_2019'
SOURCE_FILE = ETSY_ROUND + '/source/producs_2017_2019.pickle'
OUTPUT_FOLDER = ETSY_ROUND + '/output/'

In [3]:
df_source = pd.read_pickle(SOURCE_FILE)

In [4]:
#df_source.head()

In [5]:
df = df_source[['listing_id', 'materials']]

In [6]:
df.tail(2)

,listing_id,materials
user_id,,
15799995,608306837,"['leather', 'genuine leather', 'colourful leat..."
15799995,235543853,"['genuine leather', 'metal', 'leather']"


In [7]:
def removeAnyRepeatedSpace(texte):
    return re.sub(' +',' ', texte.strip())

In [8]:
# removeAnyRepeatedSpace(' stri   ng ')
# 'stri ng'

In [9]:
def removePunctuation(texte):
    exclude_punt = set(string.punctuation)
    texte = ''.join(ch for ch in texte if ch not in exclude_punt)
    return str(texte)

In [10]:
def removeAccents(texte):
    texte = unicodedata.normalize('NFD', texte)
    texte = texte.encode('ascii', 'ignore')
    texte = texte.decode("utf-8")
    return str(texte)

In [11]:
data_dict = OrderedDict()
for row in df_source.itertuples():
    index = 1
    try:
        materials = ast.literal_eval(row.materials)
        for material in materials:
            idx = '{}_{}'.format(row.listing_id, index)

            material_clean = removePunctuation(removeAnyRepeatedSpace(fix_text(material))).strip().lower()

            material_key = removeAccents(material_clean).replace(' ', '_')

            data_dict[idx] = [row.listing_id, material_key, material_clean]
            index = index + 1
    except:
        print('Error: {}'.format(row.materials))

In [12]:
# data_dict

In [13]:
pickle.dump( data_dict, open( OUTPUT_FOLDER + "data_dict.pickle", "wb" ) )

In [14]:
dic_uniques = OrderedDict()

In [15]:
for (key, value) in data_dict.items() :
    if value[2] not in dic_uniques:
        dic_uniques[value[1]] = value[2]

In [16]:
len(dic_uniques)

61097

In [19]:
# dic_uniques

In [18]:
pickle.dump( dic_uniques, open( OUTPUT_FOLDER + "dic_uniques.pickle", "wb" ) )